In [1]:
from datetime import datetime
import os

import heliopy.data.omni as omni
from matplotlib import pyplot as plt
import numpy as np
from tensorflow import keras

In [2]:
START_TIME = datetime(1995, 1, 1)
END_TIME = datetime(2018, 2, 28)
INPUT_LENGTH = 24

### 1. Load in data

In [3]:
def get_omni_rtn_data(start_time, end_time):
    identifier = 'OMNI_COHO1HR_MERGED_MAG_PLASMA'  # COHO 1HR data
    omni_data = omni._omni(start_time, end_time, identifier=identifier, intervals='yearly', warn_missing_units=False)
    return omni_data

In [4]:
data = get_omni_rtn_data(START_TIME, END_TIME).to_dataframe()

In [5]:
mag_field_strength = np.array(data["BR"])

### 2. Split into INPUT_LENGTH sections 

In [6]:
len(np.array([mag_field_strength[i:i + INPUT_LENGTH] 
                        for i in range(len(mag_field_strength) - INPUT_LENGTH)])[:, :, np.newaxis])

203014

In [7]:
inputs = np.array([mag_field_strength[i:i + INPUT_LENGTH] 
                        for i in range(len(mag_field_strength) - INPUT_LENGTH)])[:, :, np.newaxis]
outputs = np.array(mag_field_strength[INPUT_LENGTH:])

nan_check = np.array([mag_field_strength[i:i + INPUT_LENGTH + 1] 
                      for i in range(len(mag_field_strength) - INPUT_LENGTH)])

inputs = inputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]
outputs = outputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]

print("Input shape:", inputs.shape)
print("Output shape:", outputs.shape)

print("Any Nans?:", np.any(np.isnan(outputs)) or np.any(np.isnan(inputs)))

Input shape: (201389, 24, 1)
Output shape: (201389,)
Any Nans?: False


### 3. Split into train/val/test

In [8]:
# Doing this time-based, so most recent = test set, earliest = train set 
train_size = 0.7
val_size = 0.15
data_size = len(inputs)

inputs_train, outputs_train = inputs[:int(train_size * data_size)], outputs[:int(train_size * data_size)]
inputs_val, outputs_val = inputs[int(train_size * data_size):int((train_size + val_size) * data_size)], outputs[int(train_size * data_size):int((train_size + val_size) * data_size)]
inputs_test, outputs_test = inputs[int((train_size + val_size) * data_size):], outputs[int((train_size + val_size) * data_size):]

print("Train size:", len(inputs_train))
print("Val size:", len(inputs_val))
print("Test size:", len(inputs_test))

Train size: 140972
Val size: 30208
Test size: 30209


### 4. Baselines

In [10]:
baselines = {}

##### 1: Last timestep

In [20]:
for name, dset, out in zip(["last_train", "last_val", "last_test"], 
                           [inputs_train, inputs_val, inputs_test],
                           [outputs_train, outputs_val, outputs_test]):
    baselines[name] = dset[:, -1, 0]
    print("MSE {}:".format(name), np.mean((baselines[name] - out) ** 2))

MSE last_train: 3.14658
MSE last_val: 2.4640298
MSE last_test: 3.3855927


##### 2: Mean 

In [12]:
for name, dset, out in zip(["mean_train", "mean_val", "mean_test"], 
                           [inputs_train, inputs_val, inputs_test],
                           [outputs_train, outputs_val, outputs_test]):
    baselines[name] = np.mean(dset, axis=(1, 2))
    print("MSE {}:".format(name), np.mean((baselines[name] - out) ** 2))

MSE mean_train: 6.9930334
MSE mean_val: 5.7877436
MSE mean_test: 6.955669


##### 3: Median

In [13]:
for name, dset, out in zip(["median_train", "median_val", "median_test"], 
                           [inputs_train, inputs_val, inputs_test],
                           [outputs_train, outputs_val, outputs_test]):
    baselines[name] = np.median(dset, axis=(1, 2))
    print("MSE {}:".format(name), np.mean((baselines[name] - out) ** 2))

MSE median_train: 7.735584
MSE median_val: 6.426136
MSE median_test: 7.6656737


##### 4: Start

In [21]:
for name, dset, out in zip(["start_train", "start_val", "start_test"], 
                           [inputs_train, inputs_val, inputs_test],
                           [outputs_train, outputs_val, outputs_test]):
    baselines[name] = dset[:, 0, 0]
    print("MSE {}:".format(name), np.mean((baselines[name] - out) ** 2))

MSE start_train: 15.318962
MSE start_val: 12.723228
MSE start_test: 15.642863


### 5. Initial LSTM 

In [ ]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(20, activation="relu", name="lstm_initial", input_shape=(None, 1), return_sequences=True),
        keras.layers.LSTM(20, activation="relu", name="lstm_second"),
        keras.layers.Dense(1, name="dense_final", activation="linear"),
    ]
)
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train, outputs_train, validation_data=(inputs_val, outputs_val),
          batch_size=32, epochs=30, 
          callbacks=keras.callbacks.EarlyStopping(restore_best_weights=True, patience=30))

##### Test set evaluation

In [ ]:
model.evaluate(inputs_test, outputs_test)

### 6. Repeat above but use more variables

In [22]:
# Standardise all variables 
data_array = np.array(data)
data_array = (data_array - np.nanmean(data_array, axis=0)) / np.nanstd(data_array, axis=0)

In [24]:
# Split into INPUT_LENGTH sections
inputs = np.array([data_array[i:i + INPUT_LENGTH] for i in range(len(data_array) - INPUT_LENGTH)])
outputs = np.array(mag_field_strength[INPUT_LENGTH:]) # np.array(data_array[INPUT_LENGTH:, 2])

nan_check = np.array([data_array[i:i + INPUT_LENGTH + 1] for i in range(len(data_array) - INPUT_LENGTH)])

inputs = inputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]
outputs = outputs[np.where([~np.any(np.isnan(i)) for i in nan_check])]

print("Input shape:", inputs.shape)
print("Output shape:", outputs.shape)

print("Any Nans?:", np.any(np.isnan(outputs)) or np.any(np.isnan(inputs)))

Input shape: (189110, 24, 11)
Output shape: (189110,)
Any Nans?: False


In [25]:
train_size = 0.7
val_size = 0.15
data_size = len(inputs)

inputs_train, outputs_train = inputs[:int(train_size * data_size)], outputs[:int(train_size * data_size)]
inputs_val, outputs_val = inputs[int(train_size * data_size):int((train_size + val_size) * data_size)], outputs[int(train_size * data_size):int((train_size + val_size) * data_size)]
inputs_test, outputs_test = inputs[int((train_size + val_size) * data_size):], outputs[int((train_size + val_size) * data_size):]

print("Train size:", len(inputs_train))
print("Val size:", len(inputs_val))
print("Test size:", len(inputs_test))

Train size: 132377
Val size: 28366
Test size: 28367


In [ ]:
model = keras.models.Sequential(
    [
        keras.layers.LSTM(50, activation="relu", name="lstm_initial", input_shape=(None, 11), return_sequences=True),
        keras.layers.LSTM(50, activation="relu", name="lstm_second"),
        keras.layers.Dense(10, name="dense_initial", activation="relu"),
        keras.layers.Dense(1, name="dense_final", activation="linear")
    ]
)
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train, outputs_train, validation_data=(inputs_val, outputs_val),
          batch_size=32, epochs=30, 
          callbacks=keras.callbacks.EarlyStopping(restore_best_weights=True, patience=30))

In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train, outputs_train, validation_data=(inputs_val, outputs_val),
          batch_size=32, epochs=30, 
          callbacks=keras.callbacks.EarlyStopping(restore_best_weights=True, patience=30))

In [ ]:
optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.fit(inputs_train, outputs_train, validation_data=(inputs_val, outputs_val),
          batch_size=32, epochs=500, 
          callbacks=keras.callbacks.EarlyStopping(restore_best_weights=True, patience=30))

In [ ]:
model.evaluate(inputs_test, outputs_test)

In [ ]:
model.predict(inputs_test)